In [171]:
# Uncomment to install libraries if not previously installed 
# %pip install beautifulsoup4 
# %pip install requests


In [172]:
# Import libraries to build scraper
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [173]:
# Query base URL to make sure response is 200 and scraping is possible
url = "https://www.goodreads.com/shelf/show/fantasy?page=1"
response = requests.get(url)

# Check if the request was successful and get html content
if response.status_code == 200:
    html_content = response.text  
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")
else:
    print("Failed to retrieve the web page. Status code:", response.status_code)


In [174]:
# Retrieve total number of fantasy books on shelf
book_range = soup.find("div", class_="mediumText").get_text().strip()
total_books = int(book_range.split()[-1].replace(',', ''))


In [175]:
# Calculate total number of pages based on items per page and total books
items_per_page = 50  
total_pages = (total_books + items_per_page - 1) // items_per_page
#Used to set maximum number of pages to scrape
max_pages_to_scrape = 25


In [176]:
# Lists used to store individual book data
title = []
url_list = []
authors = []
avg_ratings = []
rating = []
year = []
pages = []


In [177]:
# Iterate through every page and store book information as defined in lists
for page in range(1, min(max_pages_to_scrape, total_pages) + 1):

    url = f"https://www.goodreads.com/shelf/show/fantasy?page={page}"
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "html.parser")
    booksOnPage = soup.find_all("div", "elementList")

    # Iterate through each book on the current page
    for book in booksOnPage:
        # Use try-except blocks to handle potential errors if elements are missing
        try:
            # Extract book details
            book_title = book.find("a", "bookTitle").text
            book_url = "https://www.goodreads.com" + \
                book.find("a", "bookTitle").get("href")
            author = book.find("a", "authorName").text
            rating_text = book.find(
                "span", "greyText smallText").text.split()
            avg_rating = rating_text[2]
            ratings = rating_text[4]
            published_year = rating_text[-1] if len(rating_text) == 9 else ""

            # Append the extracted data to their respective lists
            title.append(book_title)
            url_list.append(book_url)
            authors.append(author)
            avg_ratings.append(avg_rating)
            rating.append(ratings)
            year.append(published_year)
        except AttributeError:
            # Handle the case where an element is not found
            print(f"Skipping a book on page {page}.")


Skipping a book on page 1.
Skipping a book on page 1.
Skipping a book on page 2.
Skipping a book on page 2.
Skipping a book on page 3.
Skipping a book on page 3.
Skipping a book on page 4.
Skipping a book on page 4.
Skipping a book on page 5.
Skipping a book on page 5.
Skipping a book on page 6.
Skipping a book on page 6.
Skipping a book on page 7.
Skipping a book on page 7.
Skipping a book on page 8.
Skipping a book on page 8.
Skipping a book on page 9.
Skipping a book on page 9.
Skipping a book on page 10.
Skipping a book on page 10.
Skipping a book on page 12.
Skipping a book on page 12.
Skipping a book on page 13.
Skipping a book on page 13.
Skipping a book on page 14.
Skipping a book on page 14.
Skipping a book on page 15.
Skipping a book on page 15.
Skipping a book on page 16.
Skipping a book on page 16.
Skipping a book on page 17.
Skipping a book on page 17.
Skipping a book on page 18.
Skipping a book on page 18.
Skipping a book on page 19.
Skipping a book on page 19.
Skipping a

In [192]:

#Perform an additional request to each of the book urls to get their page# data
for url in url_list:
         
    try:
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "html.parser")

        book_details = soup.find("div", "FeaturedDetails")
        format_text = book_details.find(
            "p", {"data-testid": "pagesFormat"}).text.split()
        
        num_pages = format_text[0]
        pages.append(num_pages)
        
    except AttributeError:
            print('Page element not found')





In [195]:
#Convert to dataframe ready to export
df = pd.DataFrame({
    "Title": title,
    "URL": url_list,
    "Authors": authors,
    "Avg Ratings": avg_ratings,
    "Rating": rating,
    "Published_year": year,
    "Pages": pages
})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           1200 non-null   object
 1   URL             1200 non-null   object
 2   Authors         1200 non-null   object
 3   Avg Ratings     1200 non-null   object
 4   Rating          1200 non-null   object
 5   Published_year  1200 non-null   object
 6   Pages           1200 non-null   object
dtypes: object(7)
memory usage: 65.8+ KB


In [197]:
df.to_csv('book_data.csv')